In [1]:
import pandas as pd 
import numpy as np
import random
import datetime
import sklearn.linear_model as skl_lm
import matplotlib.pyplot as plt 
import utilities as utils
import fractionalDiff as fd
import pickle

In [2]:
inputs = pd.read_excel('../Data/candles.xlsx')
logicOutputs = pd.read_excel('../Data/bitcoin.xlsx')
tickers = ['indicator']
inputVariables = inputs.columns[2:]


In [3]:
inputs.columns[2:]

Index(['low', 'high', 'open', 'close', 'volume', 'ma', 'UpperBB', 'LowerBB'], dtype='object')

In [4]:
transformedInputs

NameError: name 'transformedInputs' is not defined

In [5]:
accuracies = []
pTrain = 0.7
window_0 = 7
nDaysInWindow = 1/24
deltaWindow = 0


cutoff = 1e-1

orders = np.linspace(.8, .8, 1)

scoresByOrderLst = []

for order in orders:
    print(f'Differencing order: {order}')

    lags = fd.findCutoff(order,cutoff,1)
    transformedInputs = fd.TransformInputs(inputs, inputVariables, order = order, thresholdVal = cutoff)[lags:]

    scoresLst = []

    
    ######################################### window loop starts #########################################
    for window in range(window_0, window_0 + deltaWindow + 1):
        print(f'Temporal window: {window}')
        trainScores = []
        testScores = []
        scoresWithTickersLst = []
        backtestDates = []
        stopDates = inputs.date[window:(inputs.date.shape[0])]
        #stopDates = [inputs.date[(inputs.date.shape[0])]]

        for ticker in tickers:
            
            for stopDate in stopDates:
                
                startDate = stopDate + datetime.timedelta(days=-(nDaysInWindow*window))

                x, y = utils.PrepareInputs(transformedInputs, logicOutputs[lags:], startDate, stopDate, ticker)
                
                nTrain = int(pTrain * x.shape[0])
                x = np.nan_to_num(x)
                
                xTrain, xTest, yTrain, yTest = utils.TestTrainSplit(x, y, nTrain)
    
                if len(np.unique(yTrain)) < 2:
                    continue
                
                lrm = skl_lm.LogisticRegression(random_state=0, verbose = 0).fit(xTrain, yTrain)
                trainScores.append(lrm.score(xTrain, yTrain))
                testScores.append(lrm.score(xTest, yTest))            
                backtestDates.append(stopDate)
                

            scoresWithTickers = pd.DataFrame(list(zip(backtestDates, trainScores, testScores)), columns = ['stopDate', 'train', 'test'])
            scoresWithTickers['ticker'] = ticker 
            scoresWithTickersLst.append(scoresWithTickers)

    #################################################### window loop ends #######################################################################

        scores = pd.concat(scoresWithTickersLst)
        scores['window'] = window
        scoresLst.append(scores)

    scoresDf = pd.concat(scoresLst)
    scoresDf['order'] = order
    scoresByOrderLst.append(scoresDf)

scoresByOrder = pd.concat(scoresByOrderLst)

Differencing order: 0.8
Temporal window: 7


In [115]:
yTest, lrm.predict(xTrain), lrm.predict(xTest), lrm.predict(xTest[1].reshape((1, 8)))

(array([-1,  1, -1], dtype=int64),
 array([-1,  1,  1,  1], dtype=int64),
 array([1, 1, 1], dtype=int64),
 array([1], dtype=int64))

In [116]:
xTest[1].reshape((8, 1))

array([[-0.05365535],
       [ 0.19190272],
       [ 0.3969945 ],
       [ 0.42158509],
       [-0.04406998],
       [ 0.98091542],
       [ 0.73449539],
       [-0.18636924]])

In [6]:
aggScores = scoresByOrder.groupby(['order', 'window']).agg({'train':[np.median, np.std], 'test':[np.median, np.std]}).droplevel(level=[0], axis = 1).reset_index()
aggScores.columns = ['order', 'window', 'trainMedian', 'trainStd', 'testMedian', 'testStd']

aggScores

,order,window,trainMedian,trainStd,testMedian,testStd
0,0.8,7,0.75,0.134615,0.666667,0.301922


In [8]:
pickle.dump(lrm, open('../ml_models/btc_mdl_train_2022-04-15_2022-04-30_v2.sav', 'wb'))

In [9]:
candles4Backtest = pd.read_excel('../Data/candles4backtest.xlsx')

lags = fd.findCutoff(order,cutoff,1)
transformedInputs = fd.TransformInputs(candles4Backtest, inputVariables, order = order, thresholdVal = cutoff)[lags:]
transformedInputs

KeyError: 'ma'

In [ ]:
accuracies = []
pTrain = 0.7
window_0 = 7
nDaysInWindow = 1/24
deltaWindow = 0


cutoff = 1e-1

orders = np.linspace(.8, .8, 1)

scoresByOrderLst = []

for order in orders:
    print(f'Differencing order: {order}')

    lags = fd.findCutoff(order,cutoff,1)
    transformedInputs = fd.TransformInputs(inputs, candles4Backtest, order = order, thresholdVal = cutoff)[lags:]

    scoresLst = []

    
    ######################################### window loop starts #########################################
    for window in range(window_0, window_0 + deltaWindow + 1):
        print(f'Temporal window: {window}')
        trainScores = []
        testScores = []
        scoresWithTickersLst = []
        backtestDates = []
        stopDates = inputs.date[window:(inputs.date.shape[0])]
        #stopDates = [inputs.date[(inputs.date.shape[0])]]

        for ticker in tickers:
            
            for stopDate in stopDates:
                
                startDate = stopDate + datetime.timedelta(days=-(nDaysInWindow*window))

                x, y = utils.PrepareInputs(transformedInputs, logicOutputs[lags:], startDate, stopDate, ticker)
                
                nTrain = int(pTrain * x.shape[0])
                x = np.nan_to_num(x)
                
                xTrain, xTest, yTrain, yTest = utils.TestTrainSplit(x, y, nTrain)
    
                if len(np.unique(yTrain)) < 2:
                    continue
                
                lrm = skl_lm.LogisticRegression(random_state=0, verbose = 0).fit(xTrain, yTrain)
                trainScores.append(lrm.score(xTrain, yTrain))
                testScores.append(lrm.score(xTest, yTest))            
                backtestDates.append(stopDate)
                

            scoresWithTickers = pd.DataFrame(list(zip(backtestDates, trainScores, testScores)), columns = ['stopDate', 'train', 'test'])
            scoresWithTickers['ticker'] = ticker 
            scoresWithTickersLst.append(scoresWithTickers)

    #################################################### window loop ends #######################################################################

        scores = pd.concat(scoresWithTickersLst)
        scores['window'] = window
        scoresLst.append(scores)

    scoresDf = pd.concat(scoresLst)
    scoresDf['order'] = order
    scoresByOrderLst.append(scoresDf)

scoresByOrder = pd.concat(scoresByOrderLst)

In [122]:
x

array([[-0.13335182, -2.1944122 , -2.09168574,  0.49112899, -0.91283131,
        -0.99673867, -0.76390267,  0.21238611],
       [-1.180612  , -0.45338347,  0.62096807, -0.88134118,  0.41992438,
        -1.03279675, -0.59487124, -0.03765504],
       [-1.80418693, -0.30161551, -0.22833519, -2.00751759,  1.85591985,
        -1.07788184,  0.72731468, -1.80600742],
       [ 0.70680864,  0.02082764, -0.89728116,  1.61640193, -1.3646306 ,
         1.05684115,  1.08934326, -0.59130777],
       [ 1.3116778 ,  0.80299442,  1.34158434,  0.11633267, -0.95518876,
         1.435745  ,  1.17264112, -0.40065444],
       [-0.05365535,  0.19190272,  0.3969945 ,  0.42158509, -0.04406998,
         0.98091542,  0.73449539, -0.18636924],
       [ 1.04849295,  0.55672456,  0.59721334, -0.12145794,  0.13400756,
        -0.71714552, -1.74628939,  1.7210009 ]])